## Imports

In [1]:
# pip install --upgrade scikit-learn

In [2]:
import time
from IPython.display import clear_output
import numpy    as np
import pandas   as pd
import seaborn  as sb
import matplotlib.pyplot as plt
import sklearn  as skl

from sklearn import pipeline      # Pipeline
from sklearn import preprocessing # OrdinalEncoder, LabelEncoder
from sklearn import impute
from sklearn import compose
from sklearn import model_selection # train_test_split
from sklearn import metrics         # accuracy_score, balanced_accuracy_score, plot_confusion_matrix
from sklearn import set_config
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer



set_config(display='diagram') # Useful for display the pipeline

print("Pandas  ", pd.__version__)
print("Sklearn ", skl.__version__) # Try to use 0.24

Pandas   1.2.4
Sklearn  0.24.2


## Get the dataset
- **CLOUD = True**: Download dataset from Kaggle. Necesary for cloud enviroments like COLAB. **Specify your [kaggle credentials](https://www.kaggle.com/docs/api)**.
- **CLOUD = False**: Get the dataset from your local machine. **Specify the data path**.

In [3]:
CLOUD = True

if CLOUD:
    import os
    os.environ['KAGGLE_USERNAME'] = "nurlansarkhanov"
    os.environ['KAGGLE_KEY']      = "12a306a58ae66047493a6cad2acd0715"  # See https://www.kaggle.com/docs/api
#     !pip install --upgrade kaggle
#     !kaggle competitions download -c titanic
    DATA_PATH = "./titanic/"

else:
    DATA_PATH = "./titanic/"

## Load data

In [4]:
df      = pd.read_csv(DATA_PATH + "train.csv", index_col='PassengerId')
df_test = pd.read_csv(DATA_PATH + "test.csv",  index_col='PassengerId')

print("Train DataFrame:", df.shape)
print("Test DataFrame: ", df_test.shape)

Train DataFrame: (891, 11)
Test DataFrame:  (418, 10)


## Check missings

In [5]:
df.isnull().sum()

Survived      0
Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Ticket        0
Fare          0
Cabin       687
Embarked      2
dtype: int64

In [6]:
df_test.isnull().sum()

Pclass        0
Name          0
Sex           0
Age          86
SibSp         0
Parch         0
Ticket        0
Fare          1
Cabin       327
Embarked      0
dtype: int64

# Exercise 1 (2pts):
Extract the title (Mr, Mrs, ... ) from the "Name" column.

Tips:
- split(',')[1] to get the 2nd part, and remove the surnamename
- split('.')[0] to get the 1str part, and remove the name

In [24]:
# CODE HERE get_Title_from_Name funtion
# Create this function using lambda (not def)

get_Title_from_Name = lambda name: name.split(',')[1].split('.')[0].strip()

df['Title']      = df['Name'].map(get_Title_from_Name)
df_test['Title'] = df_test['Name'].map(get_Title_from_Name)

In [25]:
df['Title'] 

PassengerId
1        Mr
2       Mrs
3      Miss
4       Mrs
5        Mr
       ... 
887     Rev
888    Miss
889    Miss
890      Mr
891      Mr
Name: Title, Length: 891, dtype: object

In [9]:
df_test['Title'] 

PassengerId
892         Mr
893        Mrs
894         Mr
895         Mr
896        Mrs
         ...  
1305        Mr
1306      Dona
1307        Mr
1308        Mr
1309    Master
Name: Title, Length: 418, dtype: object

In [11]:
assert df['Title'].values[0] == "Mr"
assert df['Title'].values[1] == "Mrs"
assert df['Title'].values[2] == "Miss"

assert df_test['Title'].values[0] == "Mr"
assert df_test['Title'].values[1] == "Mrs"
assert df_test['Title'].values[414] == "Dona"

In [7]:
df.dtypes

Survived      int64
Pclass        int64
Name         object
Sex          object
Age         float64
SibSp         int64
Parch         int64
Ticket       object
Fare        float64
Cabin        object
Embarked     object
dtype: object

# Exercise 2 (1pts):
Apply the title_dictionary to get a better information about the title. You have to overwrite the Title variable.

In [12]:
title_dictionary = {
    "Capt": "Officer",
    "Col": "Officer",
    "Major": "Officer",
    "Jonkheer": "Royalty",
    "Don": "Royalty",
    "Sir" : "Royalty",
    "Dr": "Officer",
    "Rev": "Officer",
    "the Countess":"Royalty",
    "Mme": "Mrs",
    "Mlle": "Miss",
    "Ms": "Mrs",
    "Mr" : "Mr",
    "Mrs" : "Mrs",
    "Miss" : "Miss",
    "Master" : "Master",
    "Lady" : "Royalty"
}

In [12]:
df['Title'].values[886]


'Rev'

In [13]:
df["Title"] 

PassengerId
1        Mr
2       Mrs
3      Miss
4       Mrs
5        Mr
       ... 
887     Rev
888    Miss
889    Miss
890      Mr
891      Mr
Name: Title, Length: 891, dtype: object

In [14]:
# Use map to apply the prevous dict

df["Title"] = df.Title.map(title_dictionary) 
df_test["Title"] = df_test.Title.map(title_dictionary)

# YOUR CODE HERE
# raise NotImplementedError()

In [15]:
assert df['Title'].values[886] == "Officer"
assert df_test['Title'].values[417] == "Master"

# Exercise OPTINAL (0pts):
Try to extract some information from the feature **Ticket**. Search on Internet if that colum has some kind of information.

In [14]:
df_test['Ticket']

PassengerId
892                 330911
893                 363272
894                 240276
895                 315154
896                3101298
               ...        
1305             A.5. 3236
1306              PC 17758
1307    SOTON/O.Q. 3101262
1308                359309
1309                  2668
Name: Ticket, Length: 418, dtype: object

# Exercise OPTIONAL (0pts):
Try to extract some information from the feature **Cabin**. Search on Internet if that colum has some kind of information.

In [15]:
df_test['Cabin']

PassengerId
892      NaN
893      NaN
894      NaN
895      NaN
896      NaN
        ... 
1305     NaN
1306    C105
1307     NaN
1308     NaN
1309     NaN
Name: Cabin, Length: 418, dtype: object

# Preprocessing
For X data, notice that...
- We drop Survived because is the target variable
- We drop Name because we have extracted the Title: Mr, Mrs, ...
- We drop Ticket because it has no information -> see df.Ticket.nunique()
- We drop Cabin because it has a lot of missings (77% are missings)

Then, we identify **numerical** variables and **categorical** variables,

In [9]:
x = df.drop(columns=["Survived", 'Name', 'Ticket', 'Cabin']) # X DATA (WILL BE TRAIN+VALID DATA)
y = df["Survived"] # 0 = No, 1 = Yes

x_test = df_test.drop(columns=['Name', 'Ticket', 'Cabin']) # # X_TEST DATA (NEW DATA)

In [17]:
cat_vars  = ['Sex', 'Embarked', 'Title']         # x.select_dtypes(include=[object]).columns.values.tolist()
num_vars  = ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age'] # x.select_dtypes(exclude=[object]).columns.values.tolist()

print("\nNumerical features:\n", num_vars)
print("\nCategorical features:\n", cat_vars)


Numerical features:
 ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age']

Categorical features:
 ['Sex', 'Embarked', 'Title']


In [10]:
x.dtypes

Pclass        int64
Sex          object
Age         float64
SibSp         int64
Parch         int64
Fare        float64
Embarked     object
dtype: object

# Exercise 3 (2pts):
Create a **ColumnTransformer for Tree Models**. You need to create 2 pipelines (one for numerical and other for categories). Remember:
- Categorical pipeline: Some SimpleImputer -> Some Encoder
- Numerical pipeline: Some SimpleImputer -> NO Encoder

In [26]:
num_4_treeModels = pipeline.Pipeline(steps=[
  ('imputer', impute.SimpleImputer(strategy='mean', add_indicator=False)),
    ('scalar',StandardScaler()),
    ('normalize',PowerTransformer())
])

cat_4_treeModels = pipeline.Pipeline(steps=[
  ('imputer', impute.SimpleImputer(strategy='constant', fill_value='missing')),
  ('onehot', preprocessing.OneHotEncoder(handle_unknown='ignore'))
])

tree_prepro = compose.ColumnTransformer(transformers=[
    ('num', num_4_treeModels, num_vars),
    ('cat', cat_4_treeModels, cat_vars),
], remainder='drop') # Drop other vars not specified in num_vars or cat_vars

tree_prepro

# YOUR CODE HERE
# raise NotImplementedError()

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('scalar', StandardScaler()),
                                                 ('normalize',
                                                  PowerTransformer())]),
                                 ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='missing',
                                                                strategy='constant')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['Sex', 'Embarked', 'Title'])])

In [21]:
assert type(tree_prepro)      is compose._column_transformer.ColumnTransformer
assert type(num_4_treeModels) is pipeline.Pipeline
assert type(cat_4_treeModels) is pipeline.Pipeline
assert len(num_4_treeModels) == 1
assert len(cat_4_treeModels) == 2

# Exercise 4 (1pts):
1. Complete the diccionary with some Tree Models.
2. Then we put each model in a Pipeline where:
   - first is the prepocessing with the column Transformer
   - Then is the Tree model
3. Display the fullpipeline of the LGBMClassifier

In [27]:
from sklearn.tree          import DecisionTreeClassifier
from sklearn.ensemble      import RandomForestClassifier
from sklearn.ensemble      import ExtraTreesClassifier
from sklearn.ensemble      import AdaBoostClassifier
from sklearn.ensemble      import GradientBoostingClassifier
from sklearn.experimental  import enable_hist_gradient_boosting # Necesary for HistGradientBoostingClassifier
from sklearn.ensemble      import HistGradientBoostingClassifier
from xgboost               import XGBClassifier
from lightgbm              import LGBMClassifier
from catboost              import CatBoostClassifier

In [28]:

tree_classifiers = {
  "Decision Tree": DecisionTreeClassifier(),
  "Extra Trees":ExtraTreesClassifier(),
  "Random Forest":RandomForestClassifier(),
  "AdaBoost":AdaBoostClassifier(n_estimators=200, random_state=10,learning_rate=0.1),
  "Skl GBM":GradientBoostingClassifier(),
  "Skl HistGBM":GradientBoostingClassifier(),
  "XGBoost":XGBClassifier(),
  "LightGBM":LGBMClassifier(),
  "CatBoost":CatBoostClassifier()
}

# YOUR CODE HERE
# raise NotImplementedError()

tree_classifiers = {name: pipeline.make_pipeline(tree_prepro, model) for name, model in tree_classifiers.items()}

tree_classifiers["LightGBM"]

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scalar',
                                                                   StandardScaler()),
                                                                  ('normalize',
                                                                   PowerTransformer())]),
                                                  ['Pclass', 'SibSp', 'Parch',
                                                   'Fare', 'Age']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Sex', 'Embarked',
                                                   'Title'])])),
                ('lgbmclassifier', LGBMClassifier())])

In [24]:
for pipe in tree_classifiers.values():
    assert type(pipe) is pipeline.Pipeline

# Exercise 5 (3pts):
Define a simple split validation strategy with:
- 80% for train
- 20% for validation
- With stratification
- random_state=0

And train all the models in a for loop

In [29]:
x_train, x_val, y_train, y_val = model_selection.train_test_split(
    x, y,
    test_size=0.2,
    stratify = y,   
    random_state=4 )

results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [], 'Time': []})

for model_name, model in tree_classifiers.items():

    start_time = time.time()
    model.fit(x_train, y_train)
    total_time = time.time() - start_time
    
    pred = model.predict(x_val)
    
    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y_val, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y_val, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)


results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
results_ord.index += 1 
results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')

/home/nsarkhanov/anaconda3/envs/aienv/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[19:15:03] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Learning rate set to 0.008911
0:	learn: 0.6867891	total: 50.3ms	remaining: 50.3s
1:	learn: 0.6815504	total: 51.1ms	remaining: 25.5s
2:	learn: 0.6764594	total: 52ms	remaining: 17.3s
3:	learn: 0.6718439	total: 53.4ms	remaining: 13.3s
4:	learn: 0.6668757	total: 54.6ms	remaining: 10.9s
5:	learn: 0.6613973	total: 55.6ms	remaining: 9.21s
6:	learn: 0.6567769	total: 56.7ms	remaining: 8.04s
7:	learn: 0.6537519	total: 57.2ms	remaining: 7.1s
8:	learn: 0.6486683	total: 58.3ms	remaining: 6.42s
9:	learn: 0.6448223	total: 59.2ms	remaining: 5.86s
10:	learn: 0.6397800	total: 60.2ms	remaining: 5.42s
11:	learn: 0.6355646	total: 61.2ms	remaining: 5.04s
12:	learn: 0.6320736	total: 64ms	remaining: 4.86s
13:	learn: 0.6275367	total: 65.1ms	remaining: 4.58s
14:	l

243:	learn: 0.3820581	total: 372ms	remaining: 1.15s
244:	learn: 0.3818364	total: 374ms	remaining: 1.15s
245:	learn: 0.3815207	total: 375ms	remaining: 1.15s
246:	learn: 0.3810908	total: 379ms	remaining: 1.16s
247:	learn: 0.3807964	total: 381ms	remaining: 1.16s
248:	learn: 0.3803577	total: 383ms	remaining: 1.15s
249:	learn: 0.3802647	total: 384ms	remaining: 1.15s
250:	learn: 0.3800002	total: 387ms	remaining: 1.15s
251:	learn: 0.3797744	total: 389ms	remaining: 1.15s
252:	learn: 0.3796858	total: 390ms	remaining: 1.15s
253:	learn: 0.3794663	total: 400ms	remaining: 1.17s
254:	learn: 0.3792479	total: 406ms	remaining: 1.19s
255:	learn: 0.3789638	total: 408ms	remaining: 1.18s
256:	learn: 0.3788666	total: 409ms	remaining: 1.18s
257:	learn: 0.3785468	total: 410ms	remaining: 1.18s
258:	learn: 0.3782983	total: 411ms	remaining: 1.18s
259:	learn: 0.3780425	total: 418ms	remaining: 1.19s
260:	learn: 0.3777709	total: 420ms	remaining: 1.19s
261:	learn: 0.3775516	total: 421ms	remaining: 1.19s
262:	learn: 

435:	learn: 0.3514579	total: 742ms	remaining: 959ms
436:	learn: 0.3514072	total: 743ms	remaining: 958ms
437:	learn: 0.3511939	total: 745ms	remaining: 956ms
438:	learn: 0.3509688	total: 746ms	remaining: 953ms
439:	learn: 0.3509108	total: 747ms	remaining: 951ms
440:	learn: 0.3508056	total: 748ms	remaining: 949ms
441:	learn: 0.3505804	total: 750ms	remaining: 946ms
442:	learn: 0.3504928	total: 751ms	remaining: 944ms
443:	learn: 0.3503142	total: 752ms	remaining: 942ms
444:	learn: 0.3502033	total: 753ms	remaining: 940ms
445:	learn: 0.3500848	total: 755ms	remaining: 937ms
446:	learn: 0.3499526	total: 756ms	remaining: 935ms
447:	learn: 0.3498350	total: 757ms	remaining: 933ms
448:	learn: 0.3498124	total: 758ms	remaining: 930ms
449:	learn: 0.3496594	total: 761ms	remaining: 931ms
450:	learn: 0.3495238	total: 763ms	remaining: 928ms
451:	learn: 0.3494607	total: 764ms	remaining: 926ms
452:	learn: 0.3494136	total: 765ms	remaining: 924ms
453:	learn: 0.3493405	total: 767ms	remaining: 923ms
454:	learn: 

702:	learn: 0.3222815	total: 1.3s	remaining: 548ms
703:	learn: 0.3220830	total: 1.3s	remaining: 546ms
704:	learn: 0.3220655	total: 1.3s	remaining: 544ms
705:	learn: 0.3220473	total: 1.3s	remaining: 542ms
706:	learn: 0.3220436	total: 1.3s	remaining: 540ms
707:	learn: 0.3218444	total: 1.3s	remaining: 538ms
708:	learn: 0.3218190	total: 1.3s	remaining: 535ms
709:	learn: 0.3217235	total: 1.3s	remaining: 533ms
710:	learn: 0.3216977	total: 1.31s	remaining: 531ms
711:	learn: 0.3216762	total: 1.31s	remaining: 529ms
712:	learn: 0.3216198	total: 1.31s	remaining: 527ms
713:	learn: 0.3216147	total: 1.31s	remaining: 525ms
714:	learn: 0.3214713	total: 1.31s	remaining: 523ms
715:	learn: 0.3212740	total: 1.31s	remaining: 521ms
716:	learn: 0.3212358	total: 1.32s	remaining: 520ms
717:	learn: 0.3211181	total: 1.32s	remaining: 518ms
718:	learn: 0.3210923	total: 1.32s	remaining: 516ms
719:	learn: 0.3210198	total: 1.32s	remaining: 513ms
720:	learn: 0.3209445	total: 1.32s	remaining: 511ms
721:	learn: 0.320864

962:	learn: 0.2954922	total: 1.67s	remaining: 64.2ms
963:	learn: 0.2954307	total: 1.67s	remaining: 62.5ms
964:	learn: 0.2952091	total: 1.67s	remaining: 60.7ms
965:	learn: 0.2949940	total: 1.68s	remaining: 59ms
966:	learn: 0.2949428	total: 1.68s	remaining: 57.2ms
967:	learn: 0.2947151	total: 1.68s	remaining: 55.5ms
968:	learn: 0.2946972	total: 1.68s	remaining: 53.7ms
969:	learn: 0.2945204	total: 1.68s	remaining: 52ms
970:	learn: 0.2944853	total: 1.68s	remaining: 50.2ms
971:	learn: 0.2944336	total: 1.68s	remaining: 48.5ms
972:	learn: 0.2943648	total: 1.69s	remaining: 46.8ms
973:	learn: 0.2943520	total: 1.69s	remaining: 45ms
974:	learn: 0.2942818	total: 1.69s	remaining: 43.3ms
975:	learn: 0.2940766	total: 1.69s	remaining: 41.5ms
976:	learn: 0.2938717	total: 1.69s	remaining: 39.8ms
977:	learn: 0.2936667	total: 1.69s	remaining: 38.1ms
978:	learn: 0.2936204	total: 1.69s	remaining: 36.3ms
979:	learn: 0.2936120	total: 1.7s	remaining: 34.6ms
980:	learn: 0.2933645	total: 1.7s	remaining: 32.9ms
9

,Model,Accuracy,Bal Acc.,Time
1,CatBoost,87.709497,85.948617,1.968897
2,Skl GBM,85.474860,83.590250,0.185084
3,Skl HistGBM,85.474860,83.590250,0.172093
4,XGBoost,84.916201,83.945982,0.205515
5,AdaBoost,84.357542,83.491436,0.471055
6,LightGBM,84.357542,83.221344,0.237845
7,Random Forest,82.681564,81.047431,0.323979
8,Extra Trees,79.329609,77.509881,0.377925
9,Decision Tree,77.094972,76.231884,0.054854


In [26]:
assert results_ord["Accuracy"].min() > 75
assert results_ord["Bal Acc."].min() > 75
assert len(results_ord) == 9

# Exercise 6 (3pts):
Define a 10 Fold cross validation strategy with:
- With stratification
- shuffle=True
- random_state=0

And train all the models in a for loop.

Tip you can use **[cross_val_predict](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_predict.html)** for both training and predict with 

In [48]:
from sklearn.model_selection import cross_val_predict

skf = model_selection.StratifiedKFold(
    n_splits=10, shuffle=True, random_state=0)

results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [], 'Time': []})

for model_name, model in tree_classifiers.items():     
    start_time = time.time()
    model.fit(x_train, y_train)
    
    total_time = time.time() - start_time
    pred = cross_val_predict(model, x_val, y_val, cv=skf)

    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y_val, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y_val, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)

results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
results_ord.index += 1 
results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')

/home/nsarkhanov/anaconda3/envs/aienv/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/nsarkhanov/anaconda3/envs/aienv/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[19:51:55] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[19:51:55] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/nsarkhanov/anaconda3/envs/aienv/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[19:51:55] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[19:51:55] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/nsarkhanov/anaconda3/envs/aienv/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/nsarkhanov/anaconda3/envs/aienv/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

304:	learn: 0.3687339	total: 232ms	remaining: 529ms
305:	learn: 0.3683071	total: 233ms	remaining: 529ms
306:	learn: 0.3678313	total: 234ms	remaining: 528ms
307:	learn: 0.3674721	total: 234ms	remaining: 527ms
308:	learn: 0.3668709	total: 235ms	remaining: 526ms
309:	learn: 0.3664234	total: 236ms	remaining: 525ms
310:	learn: 0.3661216	total: 237ms	remaining: 524ms
311:	learn: 0.3658230	total: 237ms	remaining: 523ms
312:	learn: 0.3650631	total: 238ms	remaining: 522ms
313:	learn: 0.3646883	total: 239ms	remaining: 522ms
314:	learn: 0.3643977	total: 240ms	remaining: 521ms
315:	learn: 0.3641969	total: 240ms	remaining: 520ms
316:	learn: 0.3639100	total: 241ms	remaining: 519ms
317:	learn: 0.3633926	total: 242ms	remaining: 518ms
318:	learn: 0.3632255	total: 242ms	remaining: 517ms
319:	learn: 0.3630701	total: 243ms	remaining: 516ms
320:	learn: 0.3624751	total: 244ms	remaining: 515ms
321:	learn: 0.3621683	total: 244ms	remaining: 514ms
322:	learn: 0.3618435	total: 245ms	remaining: 513ms
323:	learn: 

535:	learn: 0.3093918	total: 410ms	remaining: 355ms
536:	learn: 0.3093832	total: 411ms	remaining: 354ms
537:	learn: 0.3092549	total: 411ms	remaining: 353ms
538:	learn: 0.3091462	total: 416ms	remaining: 356ms
539:	learn: 0.3090030	total: 417ms	remaining: 355ms
540:	learn: 0.3088996	total: 418ms	remaining: 355ms
541:	learn: 0.3086489	total: 419ms	remaining: 354ms
542:	learn: 0.3084596	total: 419ms	remaining: 353ms
543:	learn: 0.3083151	total: 420ms	remaining: 352ms
544:	learn: 0.3080155	total: 421ms	remaining: 351ms
545:	learn: 0.3077800	total: 421ms	remaining: 350ms
546:	learn: 0.3075651	total: 422ms	remaining: 350ms
547:	learn: 0.3073028	total: 423ms	remaining: 349ms
548:	learn: 0.3071880	total: 423ms	remaining: 348ms
549:	learn: 0.3068292	total: 424ms	remaining: 347ms
550:	learn: 0.3068013	total: 425ms	remaining: 346ms
551:	learn: 0.3066138	total: 426ms	remaining: 345ms
552:	learn: 0.3064677	total: 426ms	remaining: 345ms
553:	learn: 0.3063642	total: 427ms	remaining: 344ms
554:	learn: 

762:	learn: 0.2757227	total: 591ms	remaining: 183ms
763:	learn: 0.2754371	total: 592ms	remaining: 183ms
764:	learn: 0.2753378	total: 592ms	remaining: 182ms
765:	learn: 0.2751954	total: 593ms	remaining: 181ms
766:	learn: 0.2751660	total: 594ms	remaining: 180ms
767:	learn: 0.2751392	total: 594ms	remaining: 180ms
768:	learn: 0.2750148	total: 595ms	remaining: 179ms
769:	learn: 0.2748545	total: 596ms	remaining: 178ms
770:	learn: 0.2747011	total: 596ms	remaining: 177ms
771:	learn: 0.2743001	total: 597ms	remaining: 176ms
772:	learn: 0.2740943	total: 598ms	remaining: 176ms
773:	learn: 0.2740500	total: 600ms	remaining: 175ms
774:	learn: 0.2738673	total: 601ms	remaining: 174ms
775:	learn: 0.2737970	total: 602ms	remaining: 174ms
776:	learn: 0.2736387	total: 602ms	remaining: 173ms
777:	learn: 0.2735088	total: 603ms	remaining: 172ms
778:	learn: 0.2734009	total: 604ms	remaining: 171ms
779:	learn: 0.2733113	total: 605ms	remaining: 171ms
780:	learn: 0.2731173	total: 605ms	remaining: 170ms
781:	learn: 

990:	learn: 0.2478855	total: 770ms	remaining: 6.99ms
991:	learn: 0.2478341	total: 772ms	remaining: 6.23ms
992:	learn: 0.2477490	total: 773ms	remaining: 5.45ms
993:	learn: 0.2476666	total: 774ms	remaining: 4.67ms
994:	learn: 0.2476468	total: 774ms	remaining: 3.89ms
995:	learn: 0.2475529	total: 775ms	remaining: 3.11ms
996:	learn: 0.2473002	total: 775ms	remaining: 2.33ms
997:	learn: 0.2471776	total: 776ms	remaining: 1.55ms
998:	learn: 0.2469401	total: 777ms	remaining: 777us
999:	learn: 0.2467415	total: 777ms	remaining: 0us


,Model,Accuracy,Bal Acc.,Time
1,CatBoost,81.564246,79.598155,1.768772
2,LightGBM,80.446927,77.878788,0.180099
3,Decision Tree,78.212291,76.870883,0.045533
4,XGBoost,78.212291,76.600791,0.126202
5,Random Forest,77.094972,75.151515,0.296680
6,Skl GBM,77.094972,75.691700,0.153744
7,Skl HistGBM,77.094972,75.691700,0.159015
8,Extra Trees,76.536313,74.967062,0.298679
9,AdaBoost,75.418994,73.787879,0.433923


In [44]:
assert results_ord["Accuracy"].min() > 75
assert results_ord["Bal Acc."].min() > 75
assert len(results_ord) == 9

AssertionError: 

# Exercise 7.1
Train with all data the best model

In [31]:
best_model = tree_classifiers["XGBoost"]

best_model.fit(x,y)

# YOUR CODE HERE
# raise NotImplementedError()

/home/nsarkhanov/anaconda3/envs/aienv/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[19:15:52] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scalar',
                                                                   StandardScaler()),
                                                                  ('normalize',
                                                                   PowerTransformer())]),
                                                  ['Pclass', 'SibSp', 'Parch',
                                                   'Fare', 'Age']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unkn...
                               colsample_bytree=1, gamma=0, gpu_id=-1,
                               importance_type='gain',
                               interaction_constraints='',
                               learning_rate=0.300000012, max_delta_step=0,
                               max_depth=6, min_child_weight=1, missing=nan,
                               monotone_constraints='()', n_estimators=100,
                               n_jobs=4, num_parallel_tree=1, random_state=0,
                               reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
                               subsample=1, tree_method='exact',
                               validate_parameters=1, verbosity=None))])

In [42]:
tree_prepro.fit(x_test)

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('scalar', StandardScaler()),
                                                 ('normalize',
                                                  PowerTransformer())]),
                                 ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='missing',
                                                                strategy='constant')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['Sex', 'Embarked', 'Title'])])

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('scalar', StandardScaler()),
                                                 ('normalize',
                                                  PowerTransformer())]),
                                 ['Pclass', 'SibSp', 'Parch', 'Fare', 'Age']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(fill_value='missing',
                                                                strategy='constant')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['Sex', 'Embarked', 'Title'])])

# Exercise 7.2 (2pts)
With your best model, generate the predicitions for test data (x_test)

In [43]:
test_pred = best_model.predict(x_test)

# YOUR CODE HERE
# raise NotImplementedError()

ValueError: Feature shape mismatch, expected: 28, got 418

In [33]:
assert len(test_pred) == 418
assert np.unique(test_pred).tolist() == [0,1]

# Exercise 7.3

Submit to kaggle.

- You can use the kaggle command line app. Check https://github.com/Kaggle/kaggle-api

In [34]:
sub = pd.DataFrame(test_pred, index=x_test.index, columns=["Survived"])
sub.head()

,Survived
PassengerId,
892,0
893,0
894,0
895,0
896,1


In [35]:
sub.to_csv("sub.csv")

In [38]:
!kaggle competitions submit -c titanic -f sub.csv -m "My submission message"

100%|██████████████████████████████████████| 2.77k/2.77k [00:02<00:00, 1.03kB/s]
Successfully submitted to Titanic - Machine Learning from Disaster